# Installations and Imports

In [ ]:
!pip install -U -q pdfplumber tiktoken openai chromaDB sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 764.4/764.4 kB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 470.2/470.2 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.8 MB/s eta

In [ ]:
# Install the sentence transformers library
!pip install -q -u sentence-transformers


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: -u


In [ ]:
!pip install chromadb

In [ ]:
import pdfplumber
from pathlib import Path
import pandas as pd
import numpy as np
import chromadb
from operator import itemgetter
from sentence_transformers import CrossEncoder, util
import json
import tiktoken
import openai
import chromadb

In [ ]:
from sentence_transformers import SentenceTransformer, util
# Import the SentenceTransformer library
from sentence_transformers import SentenceTransformer

# Data Loading and Handling

In [ ]:
path="Principal-Sample-Life-Insurance-Policy.pdf"

In [ ]:
# Open the PDF file
with pdfplumber.open(path) as pdf:

    # Get one of the pages from the PDF and examine it
    single_page = pdf.pages[6]

    # Extract text from the first page
    text = single_page.extract_text()

    # Extract tables from the first page
    tables = single_page.extract_tables()

    # Print the extracted text
    print(text)

Section A – Eligibility
Member Life Insurance Article 1
Member Accidental Death and Dismemberment Insurance Article 2
Dependent Life Insurance Article 3
Section B - Effective Dates
Member Life Insurance Article 1
Member Accidental Death and Dismemberment Insurance Article 2
Dependent Life Insurance Article 3
Section C - Individual Terminations
Member Life Insurance Article 1
Member Accidental Death and Dismemberment Insurance Article 2
Dependent Life Insurance Article 3
Termination for Fraud Article 4
Coverage While Outside of the United States Article 5
Section D - Continuation
Member Life Insurance Article 1
Dependent Insurance - Developmentally Disabled or
Physically Handicapped Children Article 2
Section E - Reinstatement
Reinstatement Article 1
Federal Required Family and Medical Leave Act (FMLA) Article 2
Reinstatement of Coverage for a Member or Dependent When
Coverage Ends due to Living Outside of the United States Article 3
Section F - Individual Purchase Rights
Member Life In

In [ ]:
# Function to check whether a word is present in a table or not for segregation of regular text and tables

def check_bboxes(word, table_bbox):
    # Check whether word is inside a table bbox.
    l = word['x0'], word['top'], word['x1'], word['bottom']
    r = table_bbox
    return l[0] > r[0] and l[1] > r[1] and l[2] < r[2] and l[3] < r[3]

In [ ]:
# Function to extract text from a PDF file.
# 1. Declare a variable p to store the iteration of the loop that will help us store page numbers alongside the text
# 2. Declare an empty list 'full_text' to store all the text files
# 3. Use pdfplumber to open the pdf pages one by one
# 4. Find the tables and their locations in the page
# 5. Extract the text from the tables in the variable 'tables'
# 6. Extract the regular words by calling the function check_bboxes() and checking whether words are present in the table or not
# 7. Use the cluster_objects utility to cluster non-table and table words together so that they retain the same chronology as in the original PDF
# 8. Declare an empty list 'lines' to store the page text
# 9. If a text element in present in the cluster, append it to 'lines', else if a table element is present, append the table
# 10. Append the page number and all lines to full_text, and increment 'p'
# 11. When the function has iterated over all pages, return the 'full_text' list

def extract_text_from_pdf(pdf_path):
    p = 0
    full_text = []


    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            page_no = f"Page {p+1}"
            text = page.extract_text()

            tables = page.find_tables()
            table_bboxes = [i.bbox for i in tables]
            tables = [{'table': i.extract(), 'top': i.bbox[1]} for i in tables]
            non_table_words = [word for word in page.extract_words() if not any(
                [check_bboxes(word, table_bbox) for table_bbox in table_bboxes])]
            lines = []

            for cluster in pdfplumber.utils.cluster_objects(non_table_words + tables, itemgetter('top'), tolerance=5):

                if 'text' in cluster[0]:
                    try:
                        lines.append(' '.join([i['text'] for i in cluster]))
                    except KeyError:
                        pass

                elif 'table' in cluster[0]:
                    lines.append(json.dumps(cluster[0]['table']))


            full_text.append([page_no, " ".join(lines)])
            p +=1

    return full_text

In [ ]:

# Initialize an empty list to store the extracted texts and document names
data = []

# Process the PDF file
print(f"...Processing {path}")

# Call the function to extract the text from the PDF
extracted_text = extract_text_from_pdf(path)

# Convert the extracted list to a daatframe, and add a column to store document names
extracted_text_df = pd.DataFrame(extracted_text, columns=['Page No.', 'Page_Text'])

# Append the extracted text and document name to the list
data.append(extracted_text_df)

# Print a message to indicate progress
print(f"Finished processing {path}")

# Print a message to indicate all PDFs have been processed
print("PDF have been processed.")

...Processing Principal-Sample-Life-Insurance-Policy.pdf
Finished processing Principal-Sample-Life-Insurance-Policy.pdf
PDF have been processed.


In [ ]:
# print the text
data

[   Page No.                                          Page_Text
 0    Page 1  DOROTHEA GLAUSE S655 RHODE ISLAND JOHN DOE 01/...
 1    Page 2                 This page left blank intentionally
 2    Page 3  POLICY RIDER GROUP INSURANCE POLICY NO: S655 C...
 3    Page 4                 This page left blank intentionally
 4    Page 5  PRINCIPAL LIFE INSURANCE COMPANY (called The P...
 ..      ...                                                ...
 59  Page 60  I f a Dependent who was insured dies during th...
 60  Page 61  Section D - Claim Procedures Article 1 - Notic...
 61  Page 62  A claimant may request an appeal of a claim de...
 62  Page 63                 This page left blank intentionally
 63  Page 64  Principal Life Insurance Company Des Moines, I...
 
 [64 rows x 2 columns]]

In [ ]:
insurance_pdfs_data = pd.concat(data, ignore_index=True)

In [ ]:
# print Page No. and Page_Text from the dataframe
insurance_pdfs_data

,Page No.,Page_Text
0,Page 1,DOROTHEA GLAUSE S655 RHODE ISLAND JOHN DOE 01/...
1,Page 2,This page left blank intentionally
2,Page 3,POLICY RIDER GROUP INSURANCE POLICY NO: S655 C...
3,Page 4,This page left blank intentionally
4,Page 5,PRINCIPAL LIFE INSURANCE COMPANY (called The P...
...,...,...
59,Page 60,I f a Dependent who was insured dies during th...
60,Page 61,Section D - Claim Procedures Article 1 - Notic...
61,Page 62,A claimant may request an appeal of a claim de...
62,Page 63,This page left blank intentionally


In [ ]:
# Store the metadata for each page in a separate column, now we can consider the Page_No is the metadata
insurance_pdfs_data['Metadata'] = insurance_pdfs_data.apply(lambda x: {'Page_No.': x['Page No.']}, axis=1)
insurance_pdfs_data

,Page No.,Page_Text,Metadata
0,Page 1,DOROTHEA GLAUSE S655 RHODE ISLAND JOHN DOE 01/...,{'Page_No.': 'Page 1'}
1,Page 2,This page left blank intentionally,{'Page_No.': 'Page 2'}
2,Page 3,POLICY RIDER GROUP INSURANCE POLICY NO: S655 C...,{'Page_No.': 'Page 3'}
3,Page 4,This page left blank intentionally,{'Page_No.': 'Page 4'}
4,Page 5,PRINCIPAL LIFE INSURANCE COMPANY (called The P...,{'Page_No.': 'Page 5'}
...,...,...,...
59,Page 60,I f a Dependent who was insured dies during th...,{'Page_No.': 'Page 60'}
60,Page 61,Section D - Claim Procedures Article 1 - Notic...,{'Page_No.': 'Page 61'}
61,Page 62,A claimant may request an appeal of a claim de...,{'Page_No.': 'Page 62'}
62,Page 63,This page left blank intentionally,{'Page_No.': 'Page 63'}


In [ ]:
insurance_pdfs_data.Page_Text[43]

"(4) Premium will be based on the Dependent's age and the standard rate of The Principal for the policy form to be issued. b. Purchase Qualification A Dependent will qualify for individual purchase if: (1) Dependent Life Insurance, or any portion of it, terminates because he or she ceases to be a Dependent as defined in PART I; or because the Member dies, ends Active Work, or ceases to be in a class eligible for such insurance; or (2) the Dependent spouse's Dependent Life Insurance terminates as described in PART III, Section C; or (3) the Dependent spouse's or Civil Union Partner's Dependent Life Insurance terminates because of divorce or separation or termination of a Civil Union partnership from the Member; or (4) after the Dependent has been continuously insured for Dependent Life Insurance for at least five years, such insurance terminates because the Group Policy terminates, or is amended to eliminate Dependent Life Insurance, or the Member's insurance class; or (5) the Dependent

In [ ]:
insurance_pdfs_data.iloc[43,:]["Metadata"]

{'Page_No.': 'Page 44'}

In [ ]:
# Let's also check the length of all the texts as there might be some empty pages or pages with very few words that we can drop
# Text length is
insurance_pdfs_data['No_of_Words'] = insurance_pdfs_data['Page_Text'].apply(lambda x: len(x.split(' ')))

insurance_pdfs_data['Text_Length'] = insurance_pdfs_data['Page_Text'].apply(lambda x: len(x))

In [ ]:
# print the page length
insurance_pdfs_data['Text_Length']

,Text_Length
0,188
1,34
2,1468
3,34
4,709
...,...
59,1670
60,2481
61,1891
62,34


In [ ]:
max(insurance_pdfs_data['Text_Length'])

2701

In [ ]:
max(insurance_pdfs_data['No_of_Words'])

462

In [ ]:
np.mean(insurance_pdfs_data['Text_Length'])

np.float64(1554.34375)

In [ ]:
np.mean(insurance_pdfs_data['No_of_Words'])

np.float64(263.625)

In [ ]:
# print the dataframe
insurance_pdfs_data

,Page No.,Page_Text,Metadata,No_of_Words,Text_Length
0,Page 1,DOROTHEA GLAUSE S655 RHODE ISLAND JOHN DOE 01/...,{'Page_No.': 'Page 1'},30,188
1,Page 2,This page left blank intentionally,{'Page_No.': 'Page 2'},5,34
2,Page 3,POLICY RIDER GROUP INSURANCE POLICY NO: S655 C...,{'Page_No.': 'Page 3'},230,1468
3,Page 4,This page left blank intentionally,{'Page_No.': 'Page 4'},5,34
4,Page 5,PRINCIPAL LIFE INSURANCE COMPANY (called The P...,{'Page_No.': 'Page 5'},110,709
...,...,...,...,...,...
59,Page 60,I f a Dependent who was insured dies during th...,{'Page_No.': 'Page 60'},285,1670
60,Page 61,Section D - Claim Procedures Article 1 - Notic...,{'Page_No.': 'Page 61'},418,2481
61,Page 62,A claimant may request an appeal of a claim de...,{'Page_No.': 'Page 62'},322,1891
62,Page 63,This page left blank intentionally,{'Page_No.': 'Page 63'},5,34


In [ ]:
insurance_pdfs_data.shape

(64, 5)

In [ ]:
# Retain only the rows with a text length greater than 5
insurance_pdfs_data = insurance_pdfs_data.loc[insurance_pdfs_data['No_of_Words'] > 5]
insurance_pdfs_data.shape

(61, 5)

# Chunking

In [ ]:
# Iterating over all page titles to create the final df with individual chunks
page_nos = insurance_pdfs_data["Page No."]
page_nos

,Page No.
0,Page 1
2,Page 3
4,Page 5
5,Page 6
6,Page 7
...,...
58,Page 59
59,Page 60
60,Page 61
61,Page 62


In [ ]:
# Function to split text into fixed-size chunks
def split_text_into_chunks(text, chunk_size):
    chunks = []
    words = text.split()  # Split the text into words

    current_chunk = []  # Store words for the current chunk
    current_chunk_word_count = 0  # Count of words in the current chunk

    for word in words:
        if current_chunk_word_count + len(word) + 1 <= chunk_size:
            current_chunk.append(word)
            current_chunk_word_count += len(word) + 1
        else:
            chunks.append(' '.join(current_chunk))
            current_chunk = [word]
            current_chunk_word_count = len(word)

    if current_chunk:
        chunks.append(' '.join(current_chunk))

    return chunks

In [ ]:
def process_page(page_no):
    page = insurance_pdfs_data[insurance_pdfs_data['Page No.'] == page_no].Page_Text.values[0]
    metadata = insurance_pdfs_data[insurance_pdfs_data['Page No.'] == page_no].Metadata.values[0]

    if page is not None:
        # setting chunk size
        chunk_size = 1000
        text_chunks = split_text_into_chunks(page, chunk_size)

        # Creating a DataFrame to store the chunks, page title and page metadata
        data = {'Title': [], 'Chunk Text': [], 'Metadata': []}

        for index, chunk in enumerate(text_chunks):
            data['Title'].append(page_no)
            data['Chunk Text'].append(chunk)
            # adding chunk no as part of metadata
            meta_dic=metadata.copy()
            meta_dic['Chunk_No.'] = index
            data['Metadata'].append(meta_dic)

        return pd.DataFrame(data)

In [ ]:
# creating a dataframe after calling process
all_dfs = []
for page_no in page_nos:
    df = process_page(page_no)
    if df is not None:
        all_dfs.append(df)

fixed_chunk_df = pd.concat(all_dfs, ignore_index=True)
fixed_chunk_df

,Title,Chunk Text,Metadata
0,Page 1,DOROTHEA GLAUSE S655 RHODE ISLAND JOHN DOE 01/...,"{'Page_No.': 'Page 1', 'Chunk_No.': 0}"
1,Page 3,POLICY RIDER GROUP INSURANCE POLICY NO: S655 C...,"{'Page_No.': 'Page 3', 'Chunk_No.': 0}"
2,Page 3,provision of such goods and/or services nor is...,"{'Page_No.': 'Page 3', 'Chunk_No.': 1}"
3,Page 5,PRINCIPAL LIFE INSURANCE COMPANY (called The P...,"{'Page_No.': 'Page 5', 'Chunk_No.': 0}"
4,Page 6,TABLE OF CONTENTS PART I - DEFINITIONS PART II...,"{'Page_No.': 'Page 6', 'Chunk_No.': 0}"
...,...,...,...
126,Page 61,"of the loss. Proof required includes the date,...","{'Page_No.': 'Page 61', 'Chunk_No.': 1}"
127,Page 61,claim. Written notification will be sent to th...,"{'Page_No.': 'Page 61', 'Chunk_No.': 2}"
128,Page 62,A claimant may request an appeal of a claim de...,"{'Page_No.': 'Page 62', 'Chunk_No.': 0}"
129,Page 62,have the Member or Dependent whose loss is the...,"{'Page_No.': 'Page 62', 'Chunk_No.': 1}"


In [ ]:
fixed_chunk_df[fixed_chunk_df["Title"]=="Page 44"]

,Title,Chunk Text,Metadata
85,Page 44,(4) Premium will be based on the Dependent's a...,"{'Page_No.': 'Page 44', 'Chunk_No.': 0}"
86,Page 44,Dependent's Life Insurance terminates because ...,"{'Page_No.': 'Page 44', 'Chunk_No.': 1}"
87,Page 44,Individual Policy Amount The amount of insuran...,"{'Page_No.': 'Page 44', 'Chunk_No.': 2}"


# Embedding

In [ ]:
# Load the embedding model
model_name = "all-MiniLM-L6-v2"
embedder = SentenceTransformer(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Function to generate embeddings for text
def generate_embeddings(texts):
    embeddings = embedder.encode(texts, convert_to_tensor=False)
    return embeddings

In [ ]:
# function to generate embedding on dataframe
def generate_embeddings_on_df(df):
  df['Embeddings'] = df['Chunk Text'].apply(lambda x: generate_embeddings([x])[0])

In [ ]:
# Create embeddings for 'Chunk Text' column on all three dataframes
generate_embeddings_on_df(fixed_chunk_df)

In [ ]:
fixed_chunk_df

,Title,Chunk Text,Metadata,Embeddings
0,Page 1,DOROTHEA GLAUSE S655 RHODE ISLAND JOHN DOE 01/...,"{'Page_No.': 'Page 1', 'Chunk_No.': 0}","[-0.025921896, 0.04777749, 0.05585775, 0.04239..."
1,Page 3,POLICY RIDER GROUP INSURANCE POLICY NO: S655 C...,"{'Page_No.': 'Page 3', 'Chunk_No.': 0}","[-0.09329457, 0.0038957002, 0.01330726, -0.046..."
2,Page 3,provision of such goods and/or services nor is...,"{'Page_No.': 'Page 3', 'Chunk_No.': 1}","[-0.081321746, 0.07805485, 0.00863663, -0.0806..."
3,Page 5,PRINCIPAL LIFE INSURANCE COMPANY (called The P...,"{'Page_No.': 'Page 5', 'Chunk_No.': 0}","[-0.06354085, 0.05870157, 0.07262772, -0.02723..."
4,Page 6,TABLE OF CONTENTS PART I - DEFINITIONS PART II...,"{'Page_No.': 'Page 6', 'Chunk_No.': 0}","[-0.05870885, 0.014550019, 0.03875295, -0.0371..."
...,...,...,...,...
126,Page 61,"of the loss. Proof required includes the date,...","{'Page_No.': 'Page 61', 'Chunk_No.': 1}","[-0.037135836, 0.15628286, 0.006343882, -0.010..."
127,Page 61,claim. Written notification will be sent to th...,"{'Page_No.': 'Page 61', 'Chunk_No.': 2}","[-0.14050877, 0.079364605, 0.08539709, -0.0227..."
128,Page 62,A claimant may request an appeal of a claim de...,"{'Page_No.': 'Page 62', 'Chunk_No.': 0}","[-0.053096622, 0.12034012, 0.07282932, -0.0581..."
129,Page 62,have the Member or Dependent whose loss is the...,"{'Page_No.': 'Page 62', 'Chunk_No.': 1}","[-0.13236547, 0.13249458, 0.085313246, -0.0462..."


In [ ]:
fixed_chunk_df[fixed_chunk_df["Title"]=="Page 8"]

,Title,Chunk Text,Metadata,Embeddings
8,Page 8,Section A - Member Life Insurance Schedule of ...,"{'Page_No.': 'Page 8', 'Chunk_No.': 0}","[-0.04392167, 0.062096532, 0.024250546, -0.005..."
9,Page 8,"policy has been updated effective January 1, 2...","{'Page_No.': 'Page 8', 'Chunk_No.': 1}","[-0.102203794, -0.012991507, 0.0325239, -0.025..."


In [ ]:
fixed_chunk_df[fixed_chunk_df["Title"]=="Page 28"]

,Title,Chunk Text,Metadata,Embeddings
47,Page 28,Section B - Effective Dates Article 1 - Member...,"{'Page_No.': 'Page 28', 'Chunk_No.': 0}","[-0.044510055, 0.030619249, 0.043208558, 0.028..."
48,Page 28,be waived for those Members who: (1) are eligi...,"{'Page_No.': 'Page 28', 'Chunk_No.': 1}","[-0.1008704, 0.05688079, 0.048787426, 0.029484..."
49,Page 28,"January 1, 2014 PART III - INDIVIDUAL REQUIREM...","{'Page_No.': 'Page 28', 'Chunk_No.': 2}","[-0.07717599, 0.01744752, 0.06628748, -0.01431..."


# Storing in ChromaDB

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Define the path where chroma collections will be stored
chroma_data_path = '/content/drive/My Drive/ChromaDBStore1000'

In [ ]:
client = chromadb.PersistentClient(path=chroma_data_path)

In [ ]:
# Create a collection to store the embeddings. Collections in Chroma are where you can store your embeddings, documents, and any additional metadata.
collection = client.get_or_create_collection(name="insurance-collection")

In [ ]:
collection.add(
    embeddings = fixed_chunk_df['Embeddings'].to_list(),
    documents = fixed_chunk_df['Chunk Text'].to_list(),
    metadatas = fixed_chunk_df['Metadata'].to_list(),
    ids = [str(i) for i in range(0, len(fixed_chunk_df['Embeddings']))]
)

In [ ]:
# get few of data by ids from collection
collection.get(
    ids = ['0','1','2'],
    include = ['embeddings', 'documents', 'metadatas']
)

{'ids': ['0', '1', '2'],
 'embeddings': array([[-0.0259219 ,  0.04777749,  0.05585775, ..., -0.04932665,
         -0.05851149,  0.02355204],
        [-0.09329457,  0.0038957 ,  0.01330726, ..., -0.03862753,
          0.04017466, -0.02398461],
        [-0.08132175,  0.07805485,  0.00863663, ..., -0.03209351,
          0.06832634, -0.07028012]]),
 'documents': ['DOROTHEA GLAUSE S655 RHODE ISLAND JOHN DOE 01/01/2014 711 HIGH STREET GEORGE RI 02903 GROUP POLICY FOR: RHODE ISLAND JOHN DOE ALL MEMBERS Group Member Life Insurance Print Date: 07/16/2014',
  'POLICY RIDER GROUP INSURANCE POLICY NO: S655 COVERAGE: Life EMPLOYER: RHODE ISLAND JOHN DOE Effective on the later of the Date of Issue of this Group Policy or March 1, 2005, the following will apply to your Policy: From time to time The Principal may offer or provide certain employer groups who apply for coverage with The Principal a Financial Services Hotline and Grief Support Services or any other value added service for the employees o

In [ ]:
# create a cache collection
cache_collection = client.get_or_create_collection(name='insurance-collection-cache')

In [ ]:
# peek few of elements from cache collection
cache_collection.peek()

{'ids': [],
 'embeddings': array([], dtype=float64),
 'documents': [],
 'uris': None,
 'included': ['metadatas', 'documents', 'embeddings'],
 'data': None,
 'metadatas': []}

# Testing Semantic Search

In [ ]:
query = "what is the life insurance coverage for disability"

In [ ]:
cache_results = cache_collection.query(
    query_texts=query,
    n_results=1
)

/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:01<00:00, 68.3MiB/s]


In [ ]:
cache_results

{'ids': [[]],
 'embeddings': None,
 'documents': [[]],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[]],
 'distances': [[]]}

In [ ]:
# get result from main collection
results = collection.query(
query_texts=query,
n_results=8
)
print("Result size is : " + str(len(results.items())))
results.items()

Result size is : 8


dict_items([('ids', [['83', '81', '98', '8', '80', '104', '86', '102']]), ('embeddings', None), ('documents', [['Coverage During Disability benefit in force on the date Total Disability ceases, less any individual policy amount purchased earlier under this Article 1, and less any Accelerated Benefit payment as described in PART IV, Section A, Article 7. (4) If termination is as described in b. (4) above, the maximum amount will be the Member Life Insurance benefit in force on the date the Member ceases Active Work, less any individual policy amount purchased earlier under this Article 1, and less any Accelerated Benefit payment as described in PART IV, Section A, Article 7. Article 2 - Dependent Life Insurance a. Individual Policy If a Dependent qualifies and makes timely application, he or she may purchase an individual policy of life insurance under these terms: (1) The Dependent will not be required to submit Proof of Good Health. (2) The policy will be for life insurance only. No d

The values are list of lists. We need to store them in a flattened manner in a one to one correspondence with the keys.

In [ ]:
for key, val in results.items():
  print(key)

ids
embeddings
documents
uris
included
data
metadatas
distances


In [ ]:
# Implementing Cache in Semantic Search

# Set a threshold for cache search
threshold = 0.2

ids = []
documents = []
distances = []
metadatas = []
results_df = pd.DataFrame()


# If the distance is greater than the threshold, then return the results from the main collection.

if cache_results['distances'][0] == [] or cache_results['distances'][0][0] > threshold:
      # Query the collection against the user query and return the top 10 results
      results = collection.query(
      query_texts=query,
      n_results=8
      )

      # Store the query in cache_collection as document w.r.t to ChromaDB so that it can be embedded and searched against later
      # Store retrieved text, ids, distances and metadatas in cache_collection as metadatas, so that they can be fetched easily if a query indeed matches to a query in cache
      Keys = []
      Values = []

      size = len(results.items())

      for key, val in results.items():
        if val is None:
          continue
        for i in range(size):
          Keys.append(str(key)+str(i))
          Values.append(str(val[0][i]))


      cache_collection.add(
          documents= [query],
          ids = [query],  # Or if you want to assign integers as IDs 0,1,2,.., then you can use "len(cache_results['documents'])" as will return the no. of queries currently in the cache and assign the next digit to the new query."
          metadatas = dict(zip(Keys, Values))
      )

      print("Not found in cache. Found in main collection.")

      result_dict = {'Metadatas': results['metadatas'][0], 'Documents': results['documents'][0], 'Distances': results['distances'][0], "IDs":results["ids"][0]}
      results_df = pd.DataFrame.from_dict(result_dict)
      results_df


# If the distance is, however, less than the threshold, you can return the results from cache

elif cache_results['distances'][0][0] <= threshold:
      cache_result_dict = cache_results['metadatas'][0][0]
      cache_result_dict=dict(sorted(cache_result_dict.items()))

      # Loop through each inner list and then through the dictionary
      for key, value in cache_result_dict.items():
          if 'ids' in key:
              ids.append(value)
          elif 'documents' in key:
              documents.append(value)
          elif 'distances' in key:
              distances.append(value)
          elif 'metadatas' in key:
              metadatas.append(value)

      print("Found in cache!")

      # Create a DataFrame
      results_df = pd.DataFrame({
        'IDs': ids,
        'Documents': documents,
        'Distances': distances,
        'Metadatas': metadatas
      })


Not found in cache. Found in main collection.


In [ ]:
results_df

,Metadatas,Documents,Distances,IDs
0,"{'Page_No.': 'Page 43', 'Chunk_No.': 1}",Coverage During Disability benefit in force on...,0.632907,83
1,"{'Page_No.': 'Page 42', 'Chunk_No.': 2}",Member Life Insurance or Coverage During Disab...,0.712412,81
2,"{'Chunk_No.': 0, 'Page_No.': 'Page 49'}",Payment of benefits will be subject to the Ben...,0.873322,98
3,"{'Page_No.': 'Page 8', 'Chunk_No.': 0}",Section A - Member Life Insurance Schedule of ...,0.880502,8
4,"{'Page_No.': 'Page 42', 'Chunk_No.': 1}",class eligible for insurance; or (2) after the...,0.886541,80
5,"{'Page_No.': 'Page 51', 'Chunk_No.': 0}",Coverage During Disability will cease on the e...,0.892742,104
6,"{'Page_No.': 'Page 44', 'Chunk_No.': 1}",Dependent's Life Insurance terminates because ...,0.924276,86
7,"{'Chunk_No.': 1, 'Page_No.': 'Page 50'}",effective for a qualified Member on the earlie...,0.964997,102


# Testing Cross Encoder

In [ ]:
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

In [ ]:
cross_inputs = [[query, response] for response in results_df['Documents']]
cross_rerank_scores = cross_encoder.predict(cross_inputs)

In [ ]:
cross_rerank_scores

array([ 3.1485476 ,  3.053529  ,  4.681933  ,  0.7660588 ,  1.8001738 ,
       -0.77347267,  2.6833947 ,  1.7977953 ], dtype=float32)

In [ ]:
results_df['Reranked_scores'] = cross_rerank_scores

In [ ]:
results_df

,Metadatas,Documents,Distances,IDs,Reranked_scores
0,"{'Page_No.': 'Page 43', 'Chunk_No.': 1}",Coverage During Disability benefit in force on...,0.632907,83,3.148548
1,"{'Page_No.': 'Page 42', 'Chunk_No.': 2}",Member Life Insurance or Coverage During Disab...,0.712412,81,3.053529
2,"{'Chunk_No.': 0, 'Page_No.': 'Page 49'}",Payment of benefits will be subject to the Ben...,0.873322,98,4.681933
3,"{'Page_No.': 'Page 8', 'Chunk_No.': 0}",Section A - Member Life Insurance Schedule of ...,0.880502,8,0.766059
4,"{'Page_No.': 'Page 42', 'Chunk_No.': 1}",class eligible for insurance; or (2) after the...,0.886541,80,1.800174
5,"{'Page_No.': 'Page 51', 'Chunk_No.': 0}",Coverage During Disability will cease on the e...,0.892742,104,-0.773473
6,"{'Page_No.': 'Page 44', 'Chunk_No.': 1}",Dependent's Life Insurance terminates because ...,0.924276,86,2.683395
7,"{'Chunk_No.': 1, 'Page_No.': 'Page 50'}",effective for a qualified Member on the earlie...,0.964997,102,1.797795


In [ ]:
# Return the top 3 results from semantic search

top_3_semantic = results_df.sort_values(by='Distances')
top_3_semantic[:3]

,Metadatas,Documents,Distances,IDs,Reranked_scores
0,"{'Page_No.': 'Page 43', 'Chunk_No.': 1}",Coverage During Disability benefit in force on...,0.632907,83,3.148548
1,"{'Page_No.': 'Page 42', 'Chunk_No.': 2}",Member Life Insurance or Coverage During Disab...,0.712412,81,3.053529
2,"{'Chunk_No.': 0, 'Page_No.': 'Page 49'}",Payment of benefits will be subject to the Ben...,0.873322,98,4.681933


In [ ]:
# Return the top 3 results after reranking

top_3_rerank = results_df.sort_values(by='Reranked_scores', ascending=False)
top_3_rerank[:3]

,Metadatas,Documents,Distances,IDs,Reranked_scores
2,"{'Chunk_No.': 0, 'Page_No.': 'Page 49'}",Payment of benefits will be subject to the Ben...,0.873322,98,4.681933
0,"{'Page_No.': 'Page 43', 'Chunk_No.': 1}",Coverage During Disability benefit in force on...,0.632907,83,3.148548
1,"{'Page_No.': 'Page 42', 'Chunk_No.': 2}",Member Life Insurance or Coverage During Disab...,0.712412,81,3.053529


In [ ]:
top_3_RAG = top_3_rerank[["Documents", "Metadatas"]][:3]
top_3_RAG

,Documents,Metadatas
2,Payment of benefits will be subject to the Ben...,"{'Chunk_No.': 0, 'Page_No.': 'Page 49'}"
0,Coverage During Disability benefit in force on...,"{'Page_No.': 'Page 43', 'Chunk_No.': 1}"
1,Member Life Insurance or Coverage During Disab...,"{'Page_No.': 'Page 42', 'Chunk_No.': 2}"


# RAG

In [ ]:
from google.colab import userdata
openai.api_key=userdata.get('OPENAI_API_KEY')

In [ ]:
# Define the function to generate the response. Provide a comprehensive prompt that passes the user query and the top 3 results to the model

def generate_response(query, formatted_json):
    """
    Generate a response using GPT-3.5's ChatCompletion based on the user query and retrieved information.
    """
    messages = [
                {"role": "system", "content":  "You are a helpful assistant in the insurance domain who can effectively answer user queries about insurance policies and documents."},
                {"role": "user", "content": f"""You are a helpful assistant in the insurance domain who can effectively answer user queries about insurance policies and documents.

User query is given below:


'{query}'



You are given some retrieved search results from a corpus of insurance documents, with associated metadata including the source page number. These results are in JSON format and represent individual pages of an insurance document that may be relevant to the query.
Use the given results to answer the query.

The results are in JSON format below:


{formatted_json}



Each entry in the JSON array is a dictionary that contains:
- 'Documents': The actual text from the policy document. The text may also include tables in the format of a list of lists where each nested list represents a row.
- 'Metadatas': A dictionary that includes a "Page_No" key indicating the page number where the document text appears.

Follow the guidelines below when performing the task.
1. Carefully review the content in each "Documents" field to extract relevant information to answer the query.
2. If relevant policy name(s) is/are clearly mentioned in the content of the "Documents" field, then use it in the citation.
3. Use the 'Page_No' value from the 'Metadatas' dictionary as the source page number(s) as citation.
4. If you can't provide the complete answer, then provide information to help the user locate the most relevant sections in the cited documents.
5. Answer the question first, then list all citations at the end.
6. Provide accurate numerical or policy details if they are available.
7. You do not need to use all the information in the JSON array. Only choose what is relevant to the query.
8. If any document text has tables with relevant information, please reformat the tables and present them clearly in tabular format.
9. You are a customer facing assistant, so do not provide any information on internal workings. Just answer the query directly.
10. Do not guess or make assumptions beyond what is explicitly stated in the documents.
11. Do not use 'Unknown' or 'Unspecified' or 'Not Specified' as a placeholder if a recognizable policy name exists in the document text.
12. Answer the query in a clear, professional, and informative tone.

Your answer should follow this structure:

1. A clear and concise summary or direct answer
2. Any additional helpful explanation or breakdown (if needed)
3. Citations at the end, listing:
- Policy Name (from the document text)
- Page Number (from 'Page_No' in metadata)

If the query is not relevant to the document, reply that the query is irrelevant. Provide the final response as a well-formatted and easily readable text along with the citation. Always begin with the complete response followed by the citations.
"""},
              ]

    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages
    )

    #print(messages)

    return response.choices[0].message.content.split('\n')

In [ ]:
data = top_3_RAG.to_dict(orient='records')
import json
formatted_json = json.dumps(data, indent=2)
formatted_json

'[\n  {\n    "Documents": "Payment of benefits will be subject to the Beneficiary and Facility of Payment provisions of this PART IV, Section A. Article 6 - Member Life Insurance - Coverage During Disability A Member may be eligible to continue his or her Member Life and Member Accidental Death and Dismemberment Insurance and Dependent Life Insurance coverage during the Member\'s ADL Disability or Total Disability. a. Coverage Qualification To be qualified for Coverage During Disability, a Member must: (1) become ADL Disabled or Totally Disabled while insured for Member Life Insurance; and (2) become ADL Disabled or Totally Disabled prior to the attainment of age 60; and (3) remain ADL Disabled or Totally Disabled continuously; and (4) be under the regular care and attendance of a Physician; and (5) send proof of ADL Disability or Total Disability to The Principal when required; and (6) submit to Medical Examinations or Evaluations when required; and (7) return to The Principal, withou

In [ ]:
# Generate the response
#response = generate_response(query, top_3_RAG)
response = generate_response(query, formatted_json)

In [ ]:
print("\n".join(response))

### Response:

The life insurance coverage for disability allows a Member to continue their Member Life, Member Accidental Death and Dismemberment Insurance, and Dependent Life Insurance coverage during their ADL Disability or Total Disability. To qualify for Coverage During Disability, a Member must:

1. Become ADL Disabled or Totally Disabled while insured for Member Life Insurance.
2. Become ADL Disabled or Totally Disabled before reaching age 60.
3. Remain ADL Disabled or Totally Disabled continuously.
4. Be under the regular care and attendance of a Physician.
5. Send proof of ADL Disability or Total Disability to The Principal when required.
6. Submit to Medical Examinations or Evaluations when required.
7. Return to The Principal, without a claim, any individual policy amount.

In case the Total Disability ceases, the Coverage During Disability benefit in force will be applicable, with certain deductions based on individual policy amounts purchased earlier and Accelerated Benefi

In [ ]:
def search(query):

  # Set a threshold for cache search
  threshold = 0.2

  ids = []
  documents = []
  distances = []
  metadatas = []
  results_df = pd.DataFrame()

  # try to find from cache
  cache_results = cache_collection.query(
      query_texts=query,
      n_results=1
  )

  # If the distance is greater than the threshold, then return the results from the main collection.
  if cache_results['distances'][0] == [] or cache_results['distances'][0][0] > threshold:
        # Query the collection against the user query and return the top 10 results
        results = collection.query(
        query_texts=query,
        n_results=10
        )

        # Store the query in cache_collection as document w.r.t to ChromaDB so that it can be embedded and searched against later
        # Store retrieved text, ids, distances and metadatas in cache_collection as metadatas, so that they can be fetched easily if a query indeed matches to a query in cache
        Keys = []
        Values = []

        size = len(results.items())

        for key, val in results.items():
          if val is None:
            continue
          for i in range(size):
            Keys.append(str(key)+str(i))
            Values.append(str(val[0][i]))


        cache_collection.add(
            documents= [query],
            ids = [query],  # Or if you want to assign integers as IDs 0,1,2,.., then you can use "len(cache_results['documents'])" as will return the no. of queries currently in the cache and assign the next digit to the new query."
            metadatas = dict(zip(Keys, Values))
        )

        #print("Not found in cache. Found in main collection.")

        result_dict = {'Metadatas': results['metadatas'][0], 'Documents': results['documents'][0], 'Distances': results['distances'][0], "IDs":results["ids"][0]}
        results_df = pd.DataFrame.from_dict(result_dict)
        return results_df


  # If the distance is, however, less than the threshold, you can return the results from cache

  elif cache_results['distances'][0][0] <= threshold:
        cache_result_dict = cache_results['metadatas'][0][0]
        cache_result_dict=dict(sorted(cache_result_dict.items()))

        # Loop through each inner list and then through the dictionary
        for key, value in cache_result_dict.items():
            if 'ids' in key:
                ids.append(value)
            elif 'documents' in key:
                documents.append(value)
            elif 'distances' in key:
                distances.append(value)
            elif 'metadatas' in key:
                metadatas.append(value)

        #print("Found in cache!")

        # Create a DataFrame
        return pd.DataFrame({
          'IDs': ids,
          'Documents': documents,
          'Distances': distances,
          'Metadatas': metadatas
        })

In [ ]:
def apply_cross_encoder(query, df):
  cross_inputs = [[query, response] for response in df['Documents']]
  cross_rerank_scores = cross_encoder.predict(cross_inputs)
  df['Reranked_scores'] = cross_rerank_scores
  return df

In [ ]:
def get_topn(n, df):
  top_3_rerank = df.sort_values(by='Reranked_scores', ascending=False)
  return top_3_rerank[["Documents", "Metadatas"]][:n]

In [ ]:
def format_to_json(top_3_RAG):
  data = top_3_RAG.to_dict(orient='records')
  formatted_json = json.dumps(data, indent=2)
  return formatted_json

# Queries

## Query 1

In [ ]:
query = 'what is the life insurance coverage for disability'
df = search(query)
df = apply_cross_encoder(query, df)
df = get_topn(3, df)
df

,Documents,Metadatas
2,Payment of benefits will be subject to the Ben...,"{'Chunk_No.': 0, 'Page_No.': 'Page 49'}"
0,Coverage During Disability benefit in force on...,"{'Page_No.': 'Page 43', 'Chunk_No.': 1}"
1,Member Life Insurance or Coverage During Disab...,"{'Page_No.': 'Page 42', 'Chunk_No.': 2}"


In [ ]:
df=format_to_json(df)
response = generate_response(query, df)
print("\n".join(response))

**Response:**

The Member Life Insurance policy allows for coverage during disability, where a Member may be eligible to continue their Member Life and Member Accidental Death and Dismemberment Insurance, and Dependent Life Insurance coverage during their ADL Disability or Total Disability. To qualify for Coverage During Disability, the Member must meet specific criteria, including becoming ADL Disabled or Totally Disabled while insured for Member Life Insurance, being under the regular care of a Physician, and submitting required proof of disability. This coverage ceases if the Member Life Insurance or Coverage During Disability terminates under the Group Policy.

**Citations:**

- **Policy Name:** Member Life Insurance Coverage During Disability
  - **Page Number:** Page 49, Page 42


## Query 2

In [ ]:
query = 'what is the Proof of ADL Disability or Total Disability'
df = search(query)
df = apply_cross_encoder(query, df)
df = get_topn(3, df)
df

,Documents,Metadatas
2,policy issued under his or her Individual Purc...,"{'Chunk_No.': 1, 'Page_No.': 'Page 49'}"
3,The Principal may require that a ADL Disabled ...,"{'Chunk_No.': 0, 'Page_No.': 'Page 50'}"
0,"Principal has received first proof, then final...","{'Chunk_No.': 2, 'Page_No.': 'Page 49'}"


In [ ]:
jdf=format_to_json(df)
response = generate_response(query, jdf)
print("\n".join(response))

### Response:

#### Proof of ADL Disability or Total Disability:

**Answer:**
Proof of ADL Disability or Total Disability must be provided in writing to The Principal within one year from the onset of ADL Disability or Total Disability. Subsequent evidence demonstrating the continuance of ADL Disability or Total Disability is necessary when requested by The Principal. If the disability persists for two years from the date of submission of the initial proof, further proof may be requested no more than once per year. In the event of the Member's death while ADL Disabled or Totally Disabled, final proof that the disability continued until the date of death must be furnished to The Principal.

#### Citations:

1. **Principal Insurance Policy Document**
   - Page Number: Page 49

2. **Principal Insurance Policy Document**
   - Page Number: Page 50
  
---

This response provides clear information on the requirements for submitting proof of ADL Disability or Total Disability along with the co

## Query 3

In [ ]:
query = 'what is the life insurance coverage of accidental injury while driving?'
df = search(query)
df = apply_cross_encoder(query, df)
df = get_topn(3, df)
df

,Documents,Metadatas
0,Exposure Exposure to the elements will be pres...,"{'Page_No.': 'Page 55', 'Chunk_No.': 0}"
2,2 - Benefit Qualification To qualify for benef...,"{'Chunk_No.': 1, 'Page_No.': 'Page 53'}"
7,Section A – Eligibility Member Life Insurance ...,"{'Page_No.': 'Page 7', 'Chunk_No.': 0}"


In [ ]:
jdf=format_to_json(df)
response = generate_response(query, jdf)
print("\n".join(response))

The life insurance coverage for accidental injury sustained while driving includes an additional benefit of $10,000 if the Member loses their life as a result of this injury. This benefit is contingent on meeting specific qualifications such as the use of factory-installed seat belts properly fastened at the time of the accident. The certification of the seat belt position in the accident report is also required for this additional benefit payment.

Additionally, to qualify for benefit payment under the Member Accidental Death and Dismemberment Insurance, the injury must be through external, violent, and accidental means, and must be the direct and sole cause of the loss within 365 days of the injury.

Citations:
- Policy Name: Member Life Insurance, Member Accidental Death and Dismemberment Insurance
- Page Number: Page 55, Page 53
